In [ ]:
!pip3 install elasticsearch

In [1]:
import pandas as pd
import os, json
import requests

In [10]:
'''
documents = [
    {
        "_id": "id",  #
        "document_uri": "uri", #from df
        "document ": "document", # from df,
        "tags": [], # groupby uri -> concatenated lists of tags
        "annotations": [ # groupby uri -> each row is dict for the columns "text", "tags", "target", "links"
            {
                "ann_id": "id from the row"
                "tags": []
                "target": [],
                "text": str,
                "links": dict
            }, {

            }
        ]
    }
]
'''

'\ndocuments = [\n    {\n        "_id": "id",  #\n        "document_uri": "uri", #from df\n        "document ": "document", # from df,\n        "tags": [], # groupby uri -> concatenated lists of tags\n        "annotations": [ # groupby uri -> each row is dict for the columns "text", "tags", "target", "links"\n            {\n                "ann_id": "id from the row"\n                "tags": []\n                "target": [],\n                "text": str,\n                "links": dict\n            }, {\n\n            }\n        ]\n    }\n]\n'

In [2]:
import hashlib


# Utils
def generate_id(string: str):
    return hashlib.sha1(string.encode("utf-8")).hexdigest()

In [3]:
df = pd.read_json('hypothesis_v1__12-03-22.jsonl', orient='records', lines=True)


In [4]:
df

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden
0,98Ms_oJLEeqbQuNZQjVbAg,2020-04-19T14:42:17.751689+00:00,2020-04-19T14:42:17.751689+00:00,acct:stefanherzog@hypothes.is,http://rocs.hu-berlin.de/corona/docs/forecast/...,,"[models, epidemiology, forecast]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://rocs.hu-berlin.de/corona/d...,{'title': ['Forecasts by Country']},{'html': 'https://hypothes.is/a/98Ms_oJLEeqbQu...,{'display_name': 'Stefan Herzog'},False,False
1,915IIIJLEeqpiR84gpGFyQ,2020-04-19T14:42:17.192827+00:00,2020-04-19T14:42:17.192827+00:00,acct:stefanherzog@hypothes.is,https://www.reddit.com/r/BehSciResearch/commen...,,"[literature, research, curation]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.reddit.com/r/BehSciRe...,{'title': ['r/BehSciResearch - LitCovid: Curat...,{'html': 'https://hypothes.is/a/915IIIJLEeqpiR...,{'display_name': 'Stefan Herzog'},False,False
2,9yV4JIJLEeqSnPuUlknzzQ,2020-04-19T14:42:16.801220+00:00,2020-04-19T14:42:16.801220+00:00,acct:stefanherzog@hypothes.is,http://www.the100.ci/2020/03/26/crisis-researc...,,[],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://www.the100.ci/2020/03/26/c...,"{'title': ['Crisis research, fast and slow']}",{'html': 'https://hypothes.is/a/9yV4JIJLEeqSnP...,{'display_name': 'Stefan Herzog'},False,False
3,9smAKIJLEeq6HveQAipo6g,2020-04-19T14:42:16.156297+00:00,2020-04-19T14:42:16.156297+00:00,acct:stefanherzog@hypothes.is,https://www.ncbi.nlm.nih.gov/research/coronavi...,,"[literature, research, curation]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.ncbi.nlm.nih.gov/rese...,{'title': ['LitCovid - NCBI - NLM - NIH']},{'html': 'https://hypothes.is/a/9smAKIJLEeq6Hv...,{'display_name': 'Stefan Herzog'},False,False
4,9oZlroJLEeqbQSs8lamtlQ,2020-04-19T14:42:15.751624+00:00,2020-04-19T14:42:15.751624+00:00,acct:stefanherzog@hypothes.is,http://www.the100.ci/2020/03/26/crisis-researc...,,[],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://www.the100.ci/2020/03/26/c...,"{'title': ['Crisis research, fast and slow']}",{'html': 'https://hypothes.is/a/9oZlroJLEeqbQS...,{'display_name': 'Stefan Herzog'},False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44263,IqqGzNZdEeu7qW8TVqQlWA,2021-06-26T09:01:46.663954+00:00,2021-06-26T09:01:46.663954+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,2021-03-31,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/IqqGzNZdEeu7qW...,{'display_name': None},False,False
44264,GzcHitZdEeu3utsN1zIlXA,2021-06-26T09:01:34.146545+00:00,2021-06-26T09:01:34.146545+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,"Parry, H. M., Tut, G., Faustini, S., Stephens,...","[is:preprint, lang:en, COVID-19, vaccine, immu...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/GzcHitZdEeu3ut...,{'display_name': None},False,False
44265,tkTclNZcEeu7dhcV5i6TGA,2021-06-26T08:58:44.804883+00:00,2021-06-26T08:58:44.804883+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/tkTclNZcEeu7dh...,{'display_name': None},False,False
44266,p5oL9tZcEeuXK79Bg_WtoA,2021-06-26T08:58:20.188404+00:00,2021-06-26T08:58:20.188404+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,,[ann:title],Jk8bYJdN

In [25]:
df['doc_id'] = df['uri'].apply(lambda doc: generate_id(doc))


In [26]:
sample = df.sample(n=1)

In [27]:
sample.iloc[0].to_dict()

{'id': 'qz45JLvKEeqEwGdaApFroQ',
 'created': '2020-07-01T18:42:51.701396+00:00',
 'updated': '2020-07-01T18:42:51.701396+00:00',
 'user': 'acct:Marlene_Wulf@hypothes.is',
 'uri': 'https://psyarxiv.com/v36hj/',
 'text': '2020-07-01',
 'tags': ['has:date'],
 'group': 'Jk8bYJdN',
 'permissions': {'read': ['group:__world__'],
  'admin': ['acct:Marlene_Wulf@hypothes.is'],
  'update': ['acct:Marlene_Wulf@hypothes.is'],
  'delete': ['acct:Marlene_Wulf@hypothes.is']},
 'target': [{'source': 'https://psyarxiv.com/v36hj/'}],
 'document': {'title': ['A Person-Centered Approach to Profiling COVID-Related Experiences in the United States: Preliminary Findings from the Epidemic-Pandemic Impacts Inventory (EPII)']},
 'links': {'html': 'https://hypothes.is/a/qz45JLvKEeqEwGdaApFroQ',
  'incontext': 'https://hyp.is/qz45JLvKEeqEwGdaApFroQ/psyarxiv.com/v36hj/',
  'json': 'https://hypothes.is/api/annotations/qz45JLvKEeqEwGdaApFroQ'},
 'user_info': {'display_name': None},
 'flagged': False,
 'hidden': False

In [28]:
def extract_text_from_target(row):
    if row["text"] != "":
        row["text_"] = row["text"]
    elif "selector" in row["target"][0]:
        row["text_"] = [s["exact"] for s in row["target"][0]["selector"] if s["type"] == "TextQuoteSelector"][0]
    else:
        row["text_"] = ""
    return row

In [29]:
df = df.apply(lambda row: extract_text_from_target(row), axis=1)

In [30]:
df[["text", "text_"]]

,text,text_
0,,6-day forecasts of COVID-19 case counts by cou...
1,,LitCovid: Curated literature hub for tracking ...
2,,the incentives are a gallery of pure glory
3,,LitCovid is a curated literature hub for track...
4,,"Moar errors Not just sampling error, but error..."
...,...,...
44263,2021-03-31,2021-03-31
44264,"Parry, H. M., Tut, G., Faustini, S., Stephens,...","Parry, H. M., Tut, G., Faustini, S., Stephens,..."
44265,,BackgroundAge is the major risk factor for mor...
44266,,1BNT162b2 vaccination in people over 80 years ...


In [52]:
df['target__source'] = df['target'].apply(lambda target: target[0]['source'])
df['source_is_uri'] = df.apply(lambda row: row['uri'] == row['target__source'], axis=1)

In [57]:
df

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,doc_id,text_,no_title,target__source,source_is_uri
0,98Ms_oJLEeqbQuNZQjVbAg,2020-04-19T14:42:17.751689+00:00,2020-04-19T14:42:17.751689+00:00,acct:stefanherzog@hypothes.is,http://rocs.hu-berlin.de/corona/docs/forecast/...,,"[models, epidemiology, forecast]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://rocs.hu-berlin.de/corona/d...,{'title': ['Forecasts by Country']},{'html': 'https://hypothes.is/a/98Ms_oJLEeqbQu...,{'display_name': 'Stefan Herzog'},False,False,1bae7c2fba0901589c8732057a5e03123ee18657,6-day forecasts of COVID-19 case counts by cou...,False,http://rocs.hu-berlin.de/corona/docs/forecast/...,True
1,915IIIJLEeqpiR84gpGFyQ,2020-04-19T14:42:17.192827+00:00,2020-04-19T14:42:17.192827+00:00,acct:stefanherzog@hypothes.is,https://www.reddit.com/r/BehSciResearch/commen...,,"[literature, research, curation]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.reddit.com/r/BehSciRe...,{'title': ['r/BehSciResearch - LitCovid: Curat...,{'html': 'https://hypothes.is/a/915IIIJLEeqpiR...,{'display_name': 'Stefan Herzog'},False,False,d5624117db7c3567bfcbddfb1b724579fb7e1eca,LitCovid: Curated literature hub for tracking ...,False,https://www.reddit.com/r/BehSciResearch/commen...,True
2,9yV4JIJLEeqSnPuUlknzzQ,2020-04-19T14:42:16.801220+00:00,2020-04-19T14:42:16.801220+00:00,acct:stefanherzog@hypothes.is,http://www.the100.ci/2020/03/26/crisis-researc...,,[],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://www.the100.ci/2020/03/26/c...,"{'title': ['Crisis research, fast and slow']}",{'html': 'https://hypothes.is/a/9yV4JIJLEeqSnP...,{'display_name': 'Stefan Herzog'},False,False,a30f8a303e30393880dfd0689a13f06402201842,the incentives are a gallery of pure glory,False,http://www.the100.ci/2020/03/26/crisis-researc...,True
3,9smAKIJLEeq6HveQAipo6g,2020-04-19T14:42:16.156297+00:00,2020-04-19T14:42:16.156297+00:00,acct:stefanherzog@hypothes.is,https://www.ncbi.nlm.nih.gov/research/coronavi...,,"[literature, research, curation]",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.ncbi.nlm.nih.gov/rese...,{'title': ['LitCovid - NCBI - NLM - NIH']},{'html': 'https://hypothes.is/a/9smAKIJLEeq6Hv...,{'display_name': 'Stefan Herzog'},False,False,e593d53590429e8768fe9fc7bd8a95bcb23b361f,LitCovid is a curated literature hub for track...,False,https://www.ncbi.nlm.nih.gov/research/coronavi...,True
4,9oZlroJLEeqbQSs8lamtlQ,2020-04-19T14:42:15.751624+00:00,2020-04-19T14:42:15.751624+00:00,acct:stefanherzog@hypothes.is,http://www.the100.ci/2020/03/26/crisis-researc...,,[],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://www.the100.ci/2020/03/26/c...,"{'title': ['Crisis research, fast and slow']}",{'html': 'https://hypothes.is/a/9oZlroJLEeqbQS...,{'display_name': 'Stefan Herzog'},False,False,a30f8a303e30393880dfd0689a13f06402201842,"Moar errors Not just sampling error, but error...",False,http://www.the100.ci/2020/03/26/crisis-researc...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44263,IqqGzNZdEeu7qW8TVqQlWA,2021-06-26T09:01:46.663954+00:00,2021-06-26T09:01:46.663954+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,2021-03-31,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/IqqGzNZdEeu7qW...,{'display_name': None},False,False,0a70abf8eee47b640ce4f1cb47440460aafa919f,2021-03-31,False,https://poseidon01.ssrn.com/delivery.php?ID=05...,True
44264,GzcHitZdEeu3utsN1zIlXA,2021-06-26T09:01:34.146545+00:00,2021-06-26T09:01:34.146545+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,"Parry, H. M., Tut, G., Faustini, S., Steph

In [56]:
df.query('source_is_uri == False')

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,doc_id,text_,no_title,target__source,source_is_uri


In [31]:
df['target'].apply(len).value_counts()

1    44268
Name: target, dtype: int64

In [32]:
mysample = df.query('doc_id == "0599244c5ba3029c15055ea6d260e1e1ec205d12"')
mysample

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,doc_id,text_,no_title
26335,UnuAindeEeunzl_3ANtJFA,2021-02-25T11:40:55.791222+00:00,2021-02-25T11:40:55.791222+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,"Krpan, Dario. ‘To Sit Quietly in a Room Alone:...","[is:preprint, lang:en, COVID-19, stimulation, ...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/UnuAindeEeunzl...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,"Krpan, Dario. ‘To Sit Quietly in a Room Alone:...",False
26336,DR1kvnddEeuoOu_euSRYCw,2021-02-25T11:31:49.916627+00:00,2021-02-25T11:31:49.916627+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/DR1kvnddEeuoOu...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,External input is any kind of physical stimula...,False
26337,_fgEYndcEeunyxN31N88BQ,2021-02-25T11:31:24.504121+00:00,2021-02-25T11:31:24.504121+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/_fgEYndcEeunyx...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,To Sit Quietly in a Room Alone: The Psychology...,False
26338,87VMlHdcEeuEK9Nzi53S3Q,2021-02-25T11:31:07.352506+00:00,2021-02-25T11:31:07.352506+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,2021-02-24,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/87VMlHdcEeuEK9...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,2021-02-24,False
26339,yZoevHdcEeu0oI-sHngbrQ,2021-02-25T11:29:56.707726+00:00,2021-02-25T11:29:56.707726+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:doi],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/yZoevHdcEeu0oI...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,10.31234/osf.io/zpf6b,False
26340,uPrVYHdcEeucr-ODQiYAkg,2021-02-25T11:29:28.807580+00:00,2021-02-25T11:29:28.807580+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,https://twitter.com/PsyArXivBot/status/1364692...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/uPrVYHdcEeucr-...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,https://twitter.com/PsyArXivBot/status/1364692...,False


In [33]:
df['no_title'] = df["document"].apply(lambda x: 'title' not in x)
_ = df.query('no_title == True')

In [34]:
_

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,doc_id,text_,no_title
5636,5ejIXrnmEeqSOG9RUf5Zpw,2020-06-29T08:59:53.847150+00:00,2020-06-29T08:59:53.847150+00:00,acct:edampf@hypothes.is,https://zoom.us/webinar/register/8915871539767...,This webinar is now available here: https://ww...,[has:update],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://zoom.us/webinar/register/...,{},{'html': 'https://hypothes.is/a/5ejIXrnmEeqSOG...,{'display_name': None},False,False,b2471f489186b99895f4ed9093b54a725dec284e,This webinar is now available here: https://ww...,True
9831,I5J7doVBEeqVknfVGirpWA,2020-04-23T09:02:20.363690+00:00,2020-04-23T09:02:20.363690+00:00,acct:edampf@hypothes.is,http://users.ox.ac.uk/~nuff0078/Covid/index.htm,2020-04-20,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://users.ox.ac.uk/~nuff0078/C...,{},{'html': 'https://hypothes.is/a/I5J7doVBEeqVkn...,{'display_name': None},False,False,00197a6f438d282966bc3dacf84f74826bc29228,2020-04-20,True
9832,IzBTaoVBEeqlqBdweJ0tCw,2020-04-23T09:02:19.742203+00:00,2020-04-23T09:02:19.742203+00:00,acct:edampf@hypothes.is,http://users.ox.ac.uk/~nuff0078/Covid/index.htm,"Bird, S., Nielsen, B. (2020 April 20). *Now-ca...","[is:webpage, COVID-19, lang:en, UK, England, n...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://users.ox.ac.uk/~nuff0078/C...,{},{'html': 'https://hypothes.is/a/IzBTaoVBEeqlqB...,{'display_name': None},False,False,00197a6f438d282966bc3dacf84f74826bc29228,"Bird, S., Nielsen, B. (2020 April 20). *Now-ca...",True
9834,IvCo5oVBEeqTNedlJE5Aqg,2020-04-23T09:02:19.341499+00:00,2020-04-23T09:02:19.341499+00:00,acct:edampf@hypothes.is,http://users.ox.ac.uk/~nuff0078/Covid/index.htm,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://users.ox.ac.uk/~nuff0078/C...,{},{'html': 'https://hypothes.is/a/IvCo5oVBEeqTNe...,{'display_name': None},False,False,00197a6f438d282966bc3dacf84f74826bc29228,NOW-CASTING OF Covid-19 DEATHS IN ENGLISH HOSP...,True
9835,IvSkqoVBEeqWVxeumCICZw,2020-04-23T09:02:19.331728+00:00,2020-04-23T09:02:19.331728+00:00,acct:edampf@hypothes.is,http://users.ox.ac.uk/~nuff0078/Covid/index.htm,https://twitter.com/SciBeh/status/125104802511...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'http://users.ox.ac.uk/~nuff0078/C...,{},{'html': 'https://hypothes.is/a/IvSkqoVBEeqWVx...,{'display_name': None},False,False,00197a6f438d282966bc3dacf84f74826bc29228,https://twitter.com/SciBeh/status/125104802511...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37221,0zGztntkEeugMatmeSrygA,2021-03-02T14:37:33.478289+00:00,2021-03-02T14:37:33.478289+00:00,acct:zoe_ikeotuonye@hypothes.is,https://psyarxiv.com/mh69r/,2021-02-26,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/mh69r/'}],{},{'html': 'https://hypothes.is/a/0zGztntkEeugMa...,{'display_name': None},False,False,352c955fdb2047313cf913aa8ad20e838675bc2a,2021-02-26,True
37222,xXEDlHtkEeuwNasfVms3_A,2021-03-02T14:37:10.411194+00:00,2021-03-02T14:37:10.411194+00:00,acct:zoe_ikeotuonye@hypothes.is,https://psyarxiv.com/mh69r/,"Baal, S. van, Walasek, L., & Hohwy, J. (2021)....","[is:preprint, lang:en, behavioural science, pa...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/mh69r/'}],{},{'html': 'https://hypothes.is/a/xXEDlHtkEeuwNa...,{'display_name': None},False,False,352c955fdb2047313cf913aa8ad20e838675bc2a,"Baal, S. van, Walasek, L., & Hohwy, J. (2021)....",True
37223,V0xabHtkEeuwPoNaGI9z8A,2021-03-02T14:34:05.621056+00:00,2021-03-02T14:34:05.621056+00:00,acct:zoe_ikeotuonye@hypothes.is,https://psyarxiv.com/mh69r/,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/mh69r/', 'se...",{}

In [51]:
mysample

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,doc_id,text_,no_title
26335,UnuAindeEeunzl_3ANtJFA,2021-02-25T11:40:55.791222+00:00,2021-02-25T11:40:55.791222+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,"Krpan, Dario. ‘To Sit Quietly in a Room Alone:...","[is:preprint, lang:en, COVID-19, stimulation, ...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/UnuAindeEeunzl...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,"Krpan, Dario. ‘To Sit Quietly in a Room Alone:...",False
26336,DR1kvnddEeuoOu_euSRYCw,2021-02-25T11:31:49.916627+00:00,2021-02-25T11:31:49.916627+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/DR1kvnddEeuoOu...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,External input is any kind of physical stimula...,False
26337,_fgEYndcEeunyxN31N88BQ,2021-02-25T11:31:24.504121+00:00,2021-02-25T11:31:24.504121+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/_fgEYndcEeunyx...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,To Sit Quietly in a Room Alone: The Psychology...,False
26338,87VMlHdcEeuEK9Nzi53S3Q,2021-02-25T11:31:07.352506+00:00,2021-02-25T11:31:07.352506+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,2021-02-24,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/87VMlHdcEeuEK9...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,2021-02-24,False
26339,yZoevHdcEeu0oI-sHngbrQ,2021-02-25T11:29:56.707726+00:00,2021-02-25T11:29:56.707726+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,,[ann:doi],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...","[{'source': 'https://psyarxiv.com/zpf6b/', 'se...",{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/yZoevHdcEeu0oI...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,10.31234/osf.io/zpf6b,False
26340,uPrVYHdcEeucr-ODQiYAkg,2021-02-25T11:29:28.807580+00:00,2021-02-25T11:29:28.807580+00:00,acct:Poppy_Schaffner@hypothes.is,https://psyarxiv.com/zpf6b/,https://twitter.com/PsyArXivBot/status/1364692...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://psyarxiv.com/zpf6b/'}],{'title': ['To Sit Quietly in a Room Alone: Th...,{'html': 'https://hypothes.is/a/uPrVYHdcEeucr-...,{'display_name': None},False,False,0599244c5ba3029c15055ea6d260e1e1ec205d12,https://twitter.com/PsyArXivBot/status/1364692...,False


In [74]:
def generate_document(df: pd.DataFrame) -> dict:
    """

    :param df: This dataframe is actually a "grouped-by" dataframe, meaning that doc_id, uri, document and tags are the same for all rows
    :return:
    """
    assert len(df['doc_id'].unique()) == 1
    assert len(df['uri'].unique()) == 1

    if "title" in df["document"].iloc[0]:
        doc_title = df["document"].iloc[0]["title"][0]
    else:
        doc_title = df["uri"].iloc[0] # If there's not title, the title is the uri (also target__source)

    document = {
        "_id": df['doc_id'].iloc[0],
        "document_uri": df["uri"].iloc[0],
        "document": doc_title,
        "tags": [],
        "annotations": []
    }

    for i, row in df.iterrows():
        annotation = {}
        # this is at level of document (first level)
        document["tags"] += row["tags"]

        # This is at level of annotation (second level)
        annotation["text"] =  row["text_"]
        annotation["tags"] = row["tags"]
        document["annotations"].append(annotation)

    return document


generate_document(mysample)

{'_id': '0599244c5ba3029c15055ea6d260e1e1ec205d12',
 'document_uri': 'https://psyarxiv.com/zpf6b/',
 'document': 'To Sit Quietly in a Room Alone: The Psychology of Social, Material, and Sensation Seeking Input',
 'tags': ['is:preprint',
  'lang:en',
  'COVID-19',
  'stimulation',
  'self-sufficiency',
  'materialism',
  'social',
  'input',
  'contexts',
  'deprivation',
  'external input',
  'senses',
  'sensation seeking',
  'social motives',
  'consequences',
  'engagement',
  'psychology',
  'social input',
  'social deprivation',
  'ann:summary',
  'ann:title',
  'has:date',
  'ann:doi',
  'has:context'],
 'annotations': [{'text': 'Krpan, Dario. ‘To Sit Quietly in a Room Alone: The Psychology of Social, Material, and Sensation Seeking Input’. PsyArXiv, 24 February 2021. https://doi.org/10.31234/osf.io/zpf6b.\n',
   'tags': ['is:preprint',
    'lang:en',
    'COVID-19',
    'stimulation',
    'self-sufficiency',
    'materialism',
    'social',
    'input',
    'contexts',
    'dep

In [75]:
documents = []

for doc_id in df['doc_id'].unique():
    sub_df = df.query("doc_id == @doc_id")
    documents.append(generate_document(sub_df))


In [62]:

len(documents)

9001

In [78]:
with open('hypothesis_documents_v1.jsonl', 'w') as f:
    for doc in documents:
        try:
            f.write(json.dumps(doc, ensure_ascii=False) + '\n')
        except:
            print(doc)
            raise Exception

### Object to ingest into Elastic Search

In [79]:
pd.DataFrame(documents)

,_id,document_uri,document,tags,annotations
0,1bae7c2fba0901589c8732057a5e03123ee18657,http://rocs.hu-berlin.de/corona/docs/forecast/...,Forecasts by Country,"[models, epidemiology, forecast]",[{'text': '6-day forecasts of COVID-19 case co...
1,d5624117db7c3567bfcbddfb1b724579fb7e1eca,https://www.reddit.com/r/BehSciResearch/commen...,r/BehSciResearch - LitCovid: Curated literatur...,"[literature, research, curation]",[{'text': 'LitCovid: Curated literature hub fo...
2,a30f8a303e30393880dfd0689a13f06402201842,http://www.the100.ci/2020/03/26/crisis-researc...,"Crisis research, fast and slow","[is:blog, lang:en, COVID-19, research, psychol...",[{'text': 'the incentives are a gallery of pur...
3,e593d53590429e8768fe9fc7bd8a95bcb23b361f,https://www.ncbi.nlm.nih.gov/research/coronavi...,LitCovid - NCBI - NLM - NIH,"[literature, research, curation]",[{'text': 'LitCovid is a curated literature hu...
4,21ee2a5f5f60a6cf2fc48806f1957f043286b88d,https://infotagion.com/,Home - Infotagion,[],[{'text': 'Infotagion seeks to fight the disin...
...,...,...,...,...,...
8996,05ad3e5aa298a69d665a8ef3bc87363035a418b2,https://www.tga.gov.au/reporting-problems,Report a problem or side effect,"[is:webpage, lang:en, COVID-19, Australia, gov...",[{'text': 'Report a problem or side effect | T...
8997,6ff14e396e1105f3336a893962f5eda4869c26c9,https://www.bbc.co.uk/news/health-56479462,Covid vaccine: US trial of AstraZeneca jab con...,"[has:context, has:date, is:news, lang:en, COVI...",[{'text': 'https://twitter.com/HetanShah/statu...
8998,23e00b7b9d6e4ff469335cd9903942490e4ffe8a,https://www.theguardian.com/society/2021/apr/0...,Long Covid: snapshot poll finds more than 1m p...,"[has:date, is:news, lang:en, COVID-19, long co...","[{'text': '2021-04-01', 'tags': ['has:date']},..."
8999,0a70abf8eee47b640ce4f1cb47440460aafa919f,https://poseidon01.ssrn.com/delivery.php?ID=05...,Microsoft Word - Parry et al.docx,"[has:date, is:preprint, lang:en, COVID-19, vac...","[{'text': '2021-03-31', 'tags': ['has:date']},..."



Data steps:

1. data from hypothesis as DataFrame -> each row is an annotation. To get a document, we need to groupby uri (or doc_id)
2. data from hypothesis transformed into "documents": a list of documents, where one document has a subfield "annotations", where annotations are. -> OUR REFERENCE
3. objects to ingest in Elastic Search: one document => one dict => one annotation -> we want to create these objects out of (2)

 Instead of:
(2)
```
documents = [
    {
        "_id": "id",  #
        "document_uri": "uri", #from df
        "document ": "document", # from df,
        "tags": [], # groupby uri -> concatenated lists of tags
        "annotations": [ # groupby uri -> each row is dict for the columns "text", "tags", "target", "links"
            {
                "ann_id": "id from the row"
                "tags": []
                "target": [],
                "text": str,
                "links": dict
            }, {

            }
        ]
    }
]
```

we'll use (3)
document_es = {

    "_id": doc_id + "_" + ann_id, (composition of original doc_id and ann_id)
    "parent_doc_id": doc_id, (the id of the document to which this annotation belongs)
    "document_uri": as they are in the original document
    "document": as they are in the original document
    "ann_id": annotations["ann_id"],
    "tags": annotations["tags"], (not tags from the first level, for that specific annotation)
    rest of the fields of that annotation as they are
    ...

}




In [81]:
requests.get("http://localhost:9200").json()

{'name': '3f4087d1a39c',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'QFF8stEpQ8-skiG8rGAd-Q',
 'version': {'number': '8.2.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '9876968ef3c745186b94fdabd4483e01499224ef',
  'build_date': '2022-05-25T15:47:06.259735307Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [83]:
url = "http://localhost:9200"

### Indexing in Elastic Search -> TODO: create a proper index

In [86]:
from elasticsearch import Elasticsearch, helpers
from random import getrandbits

es_client = Elasticsearch(url)

document_list = []
for i in range(1,20):
    document = {'my_field': str(getrandbits(64))[:3]} # generating a random number, transform into strings and
    document_list.append(document)
    if i % 10 == 0:
        helpers.bulk(es_client, document_list, index='my_index')
        document_list = []

### Search in Elastic Search

In [89]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch()

# Create an index called "my_index"
#es.indices.create(index='my_index')

# Add some data to the index
es.index(index='my_index', id=1, body={'name': 'John Doe', 'age': 25})
es.index(index='my_index', id=2, body={'name': 'Jane Smith', 'age': 30})
es.index(index='my_index', id=3, body={'name': 'Bob Johnson', 'age': 40})



{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 14, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'my_index', '_id': 'NPTTLYcB9g_5kwuWliJa', '_score': 1.0, '_source': {'my_field': 4783497421765913410}}, {'_index': 'my_index', '_id': 'NfTTLYcB9g_5kwuWliJc', '_score': 1.0, '_source': {'my_field': 611515340784470656}}, {'_index': 'my_index', '_id': 'NvTTLYcB9g_5kwuWliJc', '_score': 1.0, '_source': {'my_field': 7601591948758105214}}, {'_index': 'my_index', '_id': 'PPTTLYcB9g_5kwuWliJc', '_score': 1.0, '_source': {'my_field': 3434233755871833250}}, {'_index': 'my_index', '_id': 'PvTTLYcB9g_5kwuW7yKW', '_score': 1.0, '_source': {'my_field': '696'}}, {'_index': 'my_index', '_id': 'P_TTLYcB9g_5kwuW7yKW', '_score': 1.0, '_source': {'my_field': '174'}}, {'_index': 'my_index', '_id': 'QPTTLYcB9g_5kwuW7yKW', '_score': 1.0, '_source': {'my_field': '177'}}, {'_index': 'my_index', '_id': 'QfTTLYcB9g_5kwuW7yKW'

In [91]:
# Perform a search for all documents in the index
search_results = es.search(index='my_index', body={'query': {'match_all': {}}}) # query language is probably NoSQL



In [92]:
# hits is the list of results that match the query
search_results

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 17, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'my_index',
    '_id': 'NPTTLYcB9g_5kwuWliJa',
    '_score': 1.0,
    '_source': {'my_field': 4783497421765913410}},
   {'_index': 'my_index',
    '_id': 'NfTTLYcB9g_5kwuWliJc',
    '_score': 1.0,
    '_source': {'my_field': 611515340784470656}},
   {'_index': 'my_index',
    '_id': 'NvTTLYcB9g_5kwuWliJc',
    '_score': 1.0,
    '_source': {'my_field': 7601591948758105214}},
   {'_index': 'my_index',
    '_id': 'PPTTLYcB9g_5kwuWliJc',
    '_score': 1.0,
    '_source': {'my_field': 3434233755871833250}},
   {'_index': 'my_index',
    '_id': 'PvTTLYcB9g_5kwuW7yKW',
    '_score': 1.0,
    '_source': {'my_field': '696'}},
   {'_index': 'my_index',
    '_id': 'P_TTLYcB9g_5kwuW7yKW',
    '_score': 1.0,
    '_source': {'my_field': '174'}},
   {'_index': 'my_index',
    '_id': 'QPTTLYcB9g_5kwuW7yKW'